In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'openai_key'

In [3]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500) 

### (1) Load data

In [4]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2024-02-28 11:56:34.424 INFO    unstructured: Reading document from string ...
2024-02-28 11:56:34.497 INFO    unstructured: Reading document ...
2024-02-28 11:56:44.609 INFO    unstructured: Reading document from string ...
2024-02-28 11:56:44.641 INFO    unstructured: Reading document ...


2

### (2) Split data to create chunks

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [6]:
len(docs)

37

In [7]:
docs[0]

Document(page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nMoneycontrol Trending Stock\n\nInfosys\xa0INE009A01021, INFY, 500209\n\nState Bank of India\xa0INE062A01020, SBIN, 500112\n\nYes Bank\xa0INE528G01027, YESBANK, 532648\n\nBank Nifty\n\nNifty 500\n\nQuotes\n\nMutual Funds\n\nCommodities\n\nFutures & Options\n\nCurrency\n\nNews\n\nCryptocurrency\n\nForum\n\nNotices\n\nVideos\n\nGlossary\n\nAll\n\nHello, LoginHello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistCredit Score₹100 Cash RewardMy FeedMy MessagesMy AlertsMy Profile My PROMy PortfolioMy WatchlistCredit Score₹100 Cash RewardMy FeedMy MessagesMy AlertsLogoutChat with UsDownload AppFollow us on:\n\nPremium\n\nMy Feed', metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'})

### (3) Create embeddings for these chunks and save them to FAISS index

In [8]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

2024-02-28 11:56:59.326 INFO    faiss.loader: Loading faiss with AVX2 support.
2024-02-28 11:56:59.327 INFO    faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2024-02-28 11:56:59.330 INFO    faiss.loader: Loading faiss.
2024-02-28 11:57:00.278 INFO    faiss.loader: Successfully loaded faiss.


In [9]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

### (4) Retrieve similar embeddings for a given question and call LLM to retrieve final answer

In [11]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000175490D2F50>, verbose=False, combine_documents_chain=MapReduceDocumentsChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000175490D2F50>, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000175490D2F50>, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackMana

In [19]:
from langchain.chains import LLMChain

query = "what is the price of Tiago iCNG?"

langchain.debug = True

# Replace 'text-davinci-002' with the model you want to use
llm = YourLLMModel()  # Instantiate your LLM model object here
prompt = {"question": "business-prompt"}

chain = LLMChain(llm=llm, prompt=prompt)

chain({"question": query}, return_only_outputs=True)#ff


NameError: name 'YourLLMModel' is not defined